# Тупой бенчмарк

In [ ]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np
# для встроенных картинок
%pylab inline
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("train.csv", encoding='cp1251')

In [9]:
test = pd.read_csv("test.csv", encoding='cp1251')

In [16]:
def prepare_data(d, cf=True):
    print ('Time')
    tmp = d['Interval'].str.split('-')
    d['int0']=tmp.apply(lambda x: int(x[0]))
    d['int1']=tmp.apply(lambda x: int(x[1][:-1]))
    del d['Interval']
    
    print ('Date')
    d.OrderDate = pd.to_datetime(d.OrderDate)
    d.Date = pd.to_datetime(d.Date)
    
    deltaT = (d.OrderDate - d.Date).dt.days.astype(int).values
    d['deltaT'] = deltaT
    
    print ('Day')
    d['Date_day'] = d.Date.dt.day
    d['Date_month'] = d.Date.dt.month
    d['Date_weekday'] = d.Date.dt.weekday
    # train['Date_year'] = train.Date.dt.year
    
    d['OrderDate_day'] = d.OrderDate.dt.day
    d['OrderDate_month'] = d.OrderDate.dt.month
    d['OrderDate_weekday'] = d.OrderDate.dt.weekday
    # train['OrderDate_year'] = train.OrderDate.dt.year
    
    del d['Date']
    del d['OrderDate']
    
    print ('Type')
    d['DeliveryType'] = d['DeliveryType'].map({'Обычная доставка': 0, 'Доставка День в День': 1})
    
    print ('Groupby')
    cols = ['ChannelID', 'ClientID', 'DeliveryType', 'prepay', 'count_edit', 'int0', 'int1', 'deltaT', 'Date_day', 'Date_month', 'Date_weekday',
       'OrderDate_day', 'OrderDate_month', 'OrderDate_weekday']
    if cf:
        cols = cols + ['CancelFlag']
    
    data = d.groupby('OrderID')[cols].first()
    
    print ('Num')
    data['num'] = d.groupby('OrderID')['GroupID'].count()
    data['sum'] = d.groupby('OrderID')['OrderCnt'].sum()
    data['num/sum'] = data['num'] / data['sum']
    
    if cf:
        y = data.pop('CancelFlag')
        return (data, y)
    return (data)

In [23]:
data2 = prepare_data(test, cf=False)

Time
Date
Day
Type
Groupby
Num


In [24]:
data, y = prepare_data(train, cf=True)

Time
Date
Day
Type
Groupby
Num


In [29]:
import lightgbm as lgb

param = {'n_estimators':1000, 'num_leaves':6, 'objective':'binary',
         'learning_rate': 0.1, 'colsample_bytree': 0.75, 'subsample': 0.75,
        'metric': 'auc'}
         
w = lgb.cv(param, lgb.Dataset(data, label=y),
           stratified=False,
           num_boost_round=1000, nfold=4, verbose_eval=100)
max(w['auc-mean'])

[100]	cv_agg's auc: 0.697344 + 0.00654333
[200]	cv_agg's auc: 0.702906 + 0.00615931
[300]	cv_agg's auc: 0.706133 + 0.0055678
[400]	cv_agg's auc: 0.707971 + 0.00560712
[500]	cv_agg's auc: 0.709498 + 0.00520707
[600]	cv_agg's auc: 0.710534 + 0.00546179
[700]	cv_agg's auc: 0.71134 + 0.00563445
[800]	cv_agg's auc: 0.712172 + 0.00550675
[900]	cv_agg's auc: 0.712418 + 0.00562642
[1000]	cv_agg's auc: 0.712741 + 0.00559418


0.7127633282147994

In [33]:
np.argmax(w['auc-mean'])

978

In [32]:
import lightgbm as lgb

param = {'n_estimators':1000, 'num_leaves':8, 'objective':'binary',
         'learning_rate': 0.1, 'colsample_bytree': 0.75, 'subsample': 0.75,
        'metric': 'auc'}
         
w = lgb.cv(param, lgb.Dataset(data, label=y),
           stratified=False,
           num_boost_round=1000, nfold=4, verbose_eval=100)
max(w['auc-mean'])

[100]	cv_agg's auc: 0.700747 + 0.0060651
[200]	cv_agg's auc: 0.705531 + 0.00576664
[300]	cv_agg's auc: 0.708824 + 0.0056635
[400]	cv_agg's auc: 0.710562 + 0.00555759
[500]	cv_agg's auc: 0.711568 + 0.0050531
[600]	cv_agg's auc: 0.712369 + 0.00498233
[700]	cv_agg's auc: 0.71297 + 0.0050313
[800]	cv_agg's auc: 0.713529 + 0.00490358
[900]	cv_agg's auc: 0.713833 + 0.0047361
[1000]	cv_agg's auc: 0.71383 + 0.00477599


0.7139605983784897

In [36]:
model = lgb.LGBMClassifier(learning_rate=0.1, num_leaves=8,
                          n_estimators=1000,
                          colsample_bytree=0.75, subsample=0.75, random_state=1)
model.fit(data, y)
a = model.predict_proba(data2)[:,1]

In [47]:
pd.DataFrame({'ID': [str(x) + ' ' for x in  data2.index], ' Score': a}).to_csv('constant.csv', index=False)